In [ ]:
# imports

import os
import requests
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Load environment variables in a file called .env

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

# Check the key

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
elif api_key.strip() != api_key:
    print("An API key was found, but it looks like it might have space or tab characters at the start or end - please remove them - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")


In [ ]:
openai = OpenAI()

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, GPT! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gpt-4o-mini", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

## OK onwards with our first project

In [ ]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [ ]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [ ]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [ ]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [ ]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acqui

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```
To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [ ]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [ ]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)
print(response.choices[0].message.content)

Oh, let me put on my thinking cap for this one... It’s 4. Shocking, I know!


## And now let's build useful messages for GPT-4o-mini, using a function

In [ ]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [ ]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a

## Time to bring it together - the API for OpenAI is very simple!

In [ ]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [ ]:
summarize("https://edwarddonner.com")

"# Summary of Edward Donner's Website\n\nEdward Donner's website presents his interests and expertise in AI, particularly in the area of large language models (LLMs). He is the co-founder and CTO of Nebula.io, a company focused on utilizing AI to help individuals discover their potential in the job market. His previous venture was a startup named untapt, which was acquired in 2021.\n\n## Notable Features:\n- **Passions and Hobbies:** Besides coding and AI, Ed enjoys DJing and amateur electronic music production.\n- **AI Applications:** The site highlights the innovative applications of LLMs in recruitment and talent management, featuring a patented matching model and a well-received platform.\n\n## News and Announcements:\n- **Upcoming Courses:**\n  - *Connecting my courses – become an LLM expert and leader* (May 28, 2025)\n  - *2025 AI Executive Briefing* (May 18, 2025)\n  - *The Complete Agentic AI Engineering Course* (April 21, 2025)\n  - *LLM Workshop – Hands-on with Agents – resou

In [ ]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [ ]:
display_summary("https://cnn.com")

# CNN Website Summary

CNN is a comprehensive news platform offering the latest updates in various categories, including:

- **US and World News**: Covers prominent events and daily occurrences impacting the US and globally.
- **Politics**: Provides insights, analysis, and breaking news related to political developments.
- **Business and Technology**: Features news on market trends, corporate announcements, and technology advancements.
- **Health**: Discusses health-related news, innovations, and wellness guidance.
- **Entertainment**: Includes updates on movies, television, celebrity events, and cultural happenings.
- **Sports**: Covers major sports events, providing scores, highlights, and analysis.
- **Science and Climate**: Reports on scientific advancements and environmental issues.

## Recent Highlights
1. **Israel-Iran Conflict**:
   - Ongoing military exchanges between Israel and Iran, with significant updates on missile strikes, evacuations, and geopolitical implications.
   - Notable statements from political leaders and military officials regarding the escalation of hostilities.
   - Concerns regarding oil prices and global energy security as tensions rise.

2. **US Domestic Issues**:
   - Coverage of major incidents, such as the impact of recent immigration policies and violent events in Minnesota.
   - Analysis on Trump’s potential military involvement in international conflicts and his recent political maneuvers.

3. **Global News**:
   - Significant updates on Russian strikes in Ukraine, along with public responses and investigations into domestic issues within the US.

CNN continues to inform its audience with live updates, in-depth articles, and multimedia content, maintaining its role as a leading source for breaking news and comprehensive coverage of key issues.

In [ ]:
display_summary("https://www.forbes.com/profile/larry-ellison/?list=rtb/")

# Summary of the Larry Ellison Website

The website provides a comprehensive profile of **Larry Ellison**, the co-founder and CTO of Oracle Corporation, highlighting his current position as the second richest person globally, with a net worth of **$250.9 billion** as of June 17, 2025. The profile details his past roles, including stepping down as CEO in 2014 after a 37-year tenure, and his significant investments and property acquisitions, like his purchase of Lanai, Hawaii.

## Recent News Highlights:

1. **Oracle Stock Surge**: On June 13, 2025, it was reported that Ellison's fortune increased by **$40 billion** following a significant rise in Oracle's share price, positioning him among the top two richest individuals in the world.
   
2. **Cloud Infrastructure Investment**: On March 17, 2025, Oracle announced plans to invest **$5 billion** in cloud infrastructure in the U.K. over the next five years to address growing demand.
   
3. **Market Changes**: Several reports indicate that Ellison's net worth fluctuated due to market reactions, including a drop related to TikTok's U.S. data hosting challenges and a notable loss during a period of stock declines.

Overall, the content paints a picture of Ellison as a powerful figure in the tech industry, closely tied to Oracle's performance and the broader economic fluctuations of the tech sector.

In [ ]:
# Step 1: Create your prompts

system_prompt = "You are an assistant whose task is to read an email (official puprpose) and suggest a subject line for the email "
user_prompt = """
Provide a sample line for below mail
To:
Embassy of Germany
Consular Section
17, Nyaya Marg, Chanakyapuri
New Delhi - 110021

Date: June 16, 2025

Dear Sirs,

This letter is to present Ms. Ananya Rao, Business Development Manager at InnovaSoft Solutions Pvt. Ltd.

The purpose of Ms. Ananya Rao’s visit to Germany is to represent InnovaSoft Solutions Pvt. Ltd. on a business development and client onboarding trip with our German partner firm.

The relevant information pertaining to this visit is as follows:

Name of applicant: Ananya Rao

Nationality: Indian

Passport #: M1234567

Date of issue of passport: January 10, 2022

Expiry date: January 09, 2032

Place of birth: Bengaluru, India

Date of birth: April 15, 1991

Occupation: Business Development Manager

Name of inviting company: TechHub GmbH, Berlin

Period of stay: July 5, 2025 – July 19, 2025

We kindly ask you to grant Ms. Ananya Rao the appropriate short-term Schengen business visa for the above-mentioned purpose. Please be advised that InnovaSoft Solutions Pvt. Ltd. will accept full financial responsibility for Ms. Ananya Rao during this visit.

Thank you.

Yours truly,
Rakesh Mehta
Director – International Affairs
InnovaSoft Solutions Pvt. Ltd.

"""

# Step 2: Make the messages list

messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": user_prompt}] # fill this in

# Step 3: Call OpenAI

response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages)

# Step 4: print the result

print(response.choices[0].message.content)

Subject: Request for Short-Term Schengen Visa for Ms. Ananya Rao - Business Visit to Germany
